In [ ]:
import pandas
import numpy as np
import yfinance as yf

## MACD (Moving Average Convergence/Divergence) oscillators

In [ ]:
# Define the ticker symbol
tickerSymbol = 'AAPL'

# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
df = tickerData.history(period='1d', start='2021-1-1', end='2023-1-1')

df.head()

In [ ]:
# Calculate Short Term Exponential Moving Average
ShortEMA = df.Close.ewm(span=12, adjust=False).mean() 
# Calculate Long Term Exponential Moving Average
LongEMA = df.Close.ewm(span=26, adjust=False).mean() 
# Calculate MACD line
MACD = ShortEMA - LongEMA
# Calculate Signal line
signal = MACD.ewm(span=9, adjust=False).mean()

In [ ]:
df['MACD'] = MACD
df['Signal Line'] = signal

In [ ]:
def buy_sell(signal):
    Buy = []
    Sell = []
    flag = -1

    for i in range(0, len(signal)):
        if signal['MACD'][i] > signal['Signal Line'][i]:
            Sell.append(np.nan)
            if flag != 1:
                Buy.append(signal['Close'][i])
                flag = 1
            else:
                Buy.append(np.nan)
        elif signal['MACD'][i] < signal['Signal Line'][i]:
            Buy.append(np.nan)
            if flag != 0:
                Sell.append(signal['Close'][i])
                flag = 0
            else:
                Sell.append(np.nan)
        else:
            Buy.append(np.nan)
            Sell.append(np.nan)

    return (Buy, Sell)

In [ ]:
a = buy_sell(df)
buy_signal = a[0]
sell_signal = a[1]
buy_signal = [x for x in buy_signal if x is not np.nan]
sell_signal = [x for x in sell_signal if x is not np.nan]

In [ ]:
print(buy_signal)
print(sell_signal)

In [ ]:
# Assume we buy and sell at the close of the day
stock_Return = []
for i in range(0, len(buy_signal)):
    stock_Return.append(sell_signal[i] - buy_signal[i])
    
print(stock_Return)

In [ ]:
print("Cumulative Return : ", np.sum(stock_Return))

## RSI (Relative Strength Index)  

In [ ]:
def calculate_RSI(data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]

    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

df['RSI'] = calculate_RSI(df['Close'], 14)

In [ ]:
def buy_sell_RSI(signal, upper=70, lower=30):
    Buy = []
    Sell = []

    for i in range(0, len(signal)):
        if signal['RSI'][i] > upper:
            Sell.append(signal['Close'][i])
            Buy.append(np.nan)
        elif signal['RSI'][i] < lower:
            Buy.append(signal['Close'][i])
            Sell.append(np.nan)
        else:
            Buy.append(np.nan)
            Sell.append(np.nan)

    return (Buy, Sell)

In [ ]:
a = buy_sell_RSI(df)
buy_signal = a[0]
sell_signal = a[1]
buy_signal = [x for x in buy_signal if x is not np.nan]
sell_signal = [x for x in sell_signal if x is not np.nan]

In [ ]:
# Assume we buy and sell at the close of the day
stock_Return = []
for i in range(0, len(buy_signal)):
    stock_Return.append(sell_signal[i] - buy_signal[i])
    
print(stock_Return)

In [ ]:
print("Cumulative Return : ", np.sum(stock_Return))